In [10]:
import simdna
from simdna import synthetic as sn
loaded_motifs = sn.LoadedEncodeMotifs(simdna.ENCODE_MOTIFS_PATH, pseudocountProb=0.001)

In [38]:
gata_pwm = loaded_motifs.getPwm("GATA_disc1").getRows()/(np.array([0.3, 0.2, 0.2, 0.3])[None,:])

In [39]:
gata_pwm

array([[  1.07559416e+00,   9.26249075e-01,   1.79839106e+00,
          4.41309083e-01],
       [  4.58928113e-01,   2.37981905e+00,   1.61339124e+00,
          2.12261693e-01],
       [  2.22083444e+00,   5.03392355e-01,   1.25000000e-03,
          7.76070653e-01],
       [  8.33333333e-04,   1.25000000e-03,   4.99625000e+00,
          8.33333333e-04],
       [  3.33083333e+00,   1.25000000e-03,   1.25000000e-03,
          8.33333333e-04],
       [  8.33333333e-04,   1.25000000e-03,   1.25000000e-03,
          3.33083333e+00],
       [  2.92559564e+00,   1.25000000e-03,   1.25000000e-03,
          4.06071023e-01],
       [  3.24273818e+00,   1.33392725e-01,   1.25000000e-03,
          8.33333333e-04],
       [  8.33333333e-04,   2.65535450e-01,   4.73196455e+00,
          8.33333333e-04],
       [  8.11308713e-01,   1.24339161e+00,   2.53839531e+00,
          8.33333333e-04]])

In [45]:
import numpy as np
np.save("weights.npy",np.log(np.array(gata_pwm))[::-1,:][:,None,:,None]/5.0)

In [46]:
!ls

EmptyBackground_seqLength-200_numSeqs-1000.simdata.gz
Gappy Conv Test.ipynb
Prep logodds weights.ipynb
config
inputs.fa.gz
labels.txt.gz
make_hdf5_yaml
model_files
motifGrammarSimulation_twoMotifs_motif1-GATA_disc1_motif2-GATA_disc1_seqLength200_numSeq1000.simdata.gz
prep_data_and_make_hdf5.sh
runs_perf-metric-auROC.db
splits
test_data.hdf5
train_data.hdf5
valid_data.hdf5
variableSpacingGrammarSimulation_motif1-GATA_disc1_motif2-GATA_disc1_seqLength200_numSeq1000.simdata.gz
weights.npy


In [47]:
np.load("weights.npy").shape

(10, 1, 4, 1)

In [48]:
np.load("weights.npy")

array([[[[-0.04182133],
         [ 0.04356856],
         [ 0.18630642],
         [-1.41801537]]],


       [[[-1.41801537],
         [-0.26520139],
         [ 0.31086809],
         [-1.41801537]]],


       [[[ 0.23528362],
         [-0.40289154],
         [-1.33692235],
         [-1.41801537]]],


       [[[ 0.21469962],
         [-1.33692235],
         [-1.33692235],
         [-0.18024544]]],


       [[[-1.41801537],
         [-1.33692235],
         [-1.33692235],
         [ 0.2406445 ]]],


       [[[ 0.2406445 ],
         [-1.33692235],
         [-1.33692235],
         [-1.41801537]]],


       [[[-1.41801537],
         [-1.33692235],
         [ 0.32173753],
         [-1.41801537]]],


       [[[ 0.1595766 ],
         [-0.13727708],
         [-1.33692235],
         [-0.05070234]]],


       [[[-0.15577234],
         [ 0.17340489],
         [ 0.09566767],
         [-0.30998707]]],


       [[[ 0.01457464],
         [-0.01532242],
         [ 0.11737848],
         [-0.16360196]]]])

In [31]:
import keras
from keras.layers.convolutional import Convolution1DFromWeightFile

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GT 750M (CNMeM is disabled, cuDNN 5005)


In [32]:
Convolution1DFromWeightFile("weights.npy")

In [52]:
from keras.models import Sequential
from keras.layers.convolutional import Convolution1DFromWeightFile
from keras.layers.pooling import MaxPoolingFilter1D
from keras.layers.core import Activation

model = Sequential()
model.add(Convolution1DFromWeightFile("weights.npy", input_shape=(200,4)))
model.add(Activation("relu"))
model.add(MaxPoolingFilter1D(pool_length=10))
model.compile(optimizer='rmsprop',
          loss='categorical_crossentropy')

In [55]:
import simdna
data = simdna.synthetic.core.read_simdata_file("variableSpacingGrammarSimulation_motif1-GATA_disc1_motif2-TAL1_known1_seqLength200_numSeq1000.simdata.gz")

In [58]:
from avutils import util

onehot_data = np.array([util.seq_to_one_hot(x) for x in data.sequences])

In [62]:
print(data.embeddings[0][2])

pos-71_GATA_disc1-CCCGATAAGG-Gap7-TAL1_known1-CCAGCCAGATGGTGGC
